<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/hurr/ATL/2025/dbuoy_nowCast_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download surface in-situ data from NESDIS iQuam
#
!wget https://www.star.nesdis.noaa.gov/pub/socd/sst/iquam/v2.10/202510-STAR-L2i_GHRSST-SST-iQuam-V2.10-v01.0-fv00.0.nc

In [ ]:
!pip install cartopy

!pip install boto3
!pip install botocore

In [ ]:
import boto3
import botocore

import glob as glob

import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [ ]:
obs_ids = {'ship': 1,
          'buoy': 2,
          'tMoor': 3,
          'cMoor': 4,
          'Argo': 5}

# Note: Here we'll use buoy and coastal moorings only.

In [ ]:
def load_iquam_data(iquam_file_path, obs_ids, obsType):
  """
  Loads iQuam data from a NetCDF file and subsets it based on:
  - Observation type.

  Args:
    iquam_file_path: Path to the iQuam NetCDF file.
    obs_ids: Dictionary mapping observation type names to their IDs.
    obsType: String representing the desired observation type (e.g., 'buoy').

  Returns:
    A tuple containing the subsetted data arrays:
    (oType_subset, year, month, day, hour, minute, pType, lat, lon, sst, qcFlag)
  """
  ds = xr.open_dataset(iquam_file_path, decode_timedelta=False)

  oType = ds.platform_type.values[:]
  oType_subset = oType[oType == obs_ids[obsType]]

  year = ds.year.values[:][oType == obs_ids[obsType]]
  month = ds.month.values[:][oType == obs_ids[obsType]]
  day = ds.day.values[:][oType == obs_ids[obsType]]
  hour = ds.hour.values[:][oType == obs_ids[obsType]]
  minute = ds.minute.values[:][oType == obs_ids[obsType]]

  pId = ds.platform_id.values[:][oType == obs_ids[obsType]]

  lat = ds.lat.values[:][oType == obs_ids[obsType]]
  lon = ds.lon.values[:][oType == obs_ids[obsType]]
  sst = ds.sst.values[:][oType == obs_ids[obsType]]
  qcFlag = ds.quality_level.values[:][oType == obs_ids[obsType]]

  #depth = None
  #if obsType == 'Argo':
  #depth = ds.depth.values[:][oType == obs_ids[obsType]]

  return (oType_subset, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag)

In [ ]:
def load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value):

  mask = qcFlag == best_quality_value

  # Apply the mask to all data arrays
  oType_filtered = oType[mask]
  year_filtered = year[mask]
  month_filtered = month[mask]
  day_filtered = day[mask]
  hour_filtered = hour[mask]
  minute_filtered = minute[mask]
  pId_filtered = pId[mask]
  lat_filtered = lat[mask]
  lon_filtered = lon[mask]
  sst_filtered = sst[mask]
  qcFlag_filtered = qcFlag[mask]

  #if obsType == 'Argo':
  #depth_filtered = depth[mask]

  return (oType_filtered, year_filtered, month_filtered, day_filtered,
          hour_filtered, minute_filtered, pId_filtered, lat_filtered,
          lon_filtered, sst_filtered, qcFlag_filtered)

In [ ]:
def filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, track_year, track_mon, track_day):
    """
    Filters iQuam data arrays based on the year, month, and day of the hurricane track.

    Args:
        oType, year, month, day, hour, minute, pType, lat, lon, sst, qcFlag: Data arrays.
        track_year, track_mon, track_day: Integer values for the year, month, and day of the track.

    Returns:
        A tuple containing the filtered data arrays.
    """
    mask = (year == track_year) & (month == track_mon) & (day == track_day)

    oType_filtered = oType[mask]
    year_filtered = year[mask]
    month_filtered = month[mask]
    day_filtered = day[mask]
    hour_filtered = hour[mask]
    minute_filtered = minute[mask]
    pId_filtered = pId[mask]
    lat_filtered = lat[mask]
    lon_filtered = lon[mask]
    sst_filtered = sst[mask]
    qcFlag_filtered = qcFlag[mask]

    return (oType_filtered, year_filtered, month_filtered, day_filtered,
            hour_filtered, minute_filtered, pId_filtered, lat_filtered,
            lon_filtered, sst_filtered, qcFlag_filtered)

In [ ]:
# Create a boolean mask based on extent of the storm and return what's within that domain
def filter_by_extent(lon_min, lon_max, lat_min, lat_max, oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag):

  mask = (lon > lon_min) & (lon < lon_max) & (lat > lat_min) & (lat < lat_max)

  # Apply the above defined mask
  filtered_oType = oType[mask]
  filtered_year = year[mask]
  filtered_month = month[mask]
  filtered_day = day[mask]
  filtered_hour = hour[mask]
  filtered_minute = minute[mask]
  filtered_pId = pId[mask]
  filtered_lat = lat[mask]
  filtered_lon = lon[mask]
  filtered_sst = sst[mask]
  filtered_qcFlag = qcFlag[mask]

  return (filtered_oType, filtered_year, filtered_month, filtered_day,
          filtered_hour, filtered_minute, filtered_pId, filtered_lat,
          filtered_lon, filtered_sst, filtered_qcFlag)

In [ ]:
def download_s3_file(BUCKET_NAME, KEY, fname):
    """
    Downloads a file from an AWS S3 bucket.

    Args:
        BUCKET_NAME (str): The name of the S3 bucket.
        KEY (str): The key (prefix) of the object in the bucket.
        fname (str): The filename of the object to download.

    Returns:
        str or None: The local filename if successful, None otherwise.
    """
    KEY_WITH_FNAME = f'{KEY}{fname}' # Construct the full key with the filename

    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))

    try:
        s3.download_file(BUCKET_NAME, KEY_WITH_FNAME, fname) # Use the constructed key and fname for local filename
        print(f"Successfully downloaded {KEY_WITH_FNAME} to {fname}")
        return fname
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print(f"The object does not exist: {KEY_WITH_FNAME}")
        else:
            print(f"An error occurred: {e}")
        return None

In [ ]:
def create_and_save_buoy_dataset(buoy_year, buoy_month, buoy_day, buoy_hour, buoy_minute, buoy_lat, buoy_lon, buoy_sst, pId, buoy_ID, save_data_path):
    """
    Creates an xarray dataset from buoy data arrays and saves it to a NetCDF file.

    Args:
        buoy_year, buoy_month, buoy_day, buoy_hour, buoy_minute, buoy_lat, buoy_lon, buoy_sst: Data arrays for a specific buoy.
        pId_masked: Platform ID values masked for the specific buoy.
        buoy_ID: ID of the buoy.
        save_data_path: Path to the directory where the dataset will be saved.

    Returns:
        An xarray Dataset containing the buoy data.
    """
    # Create a time coordinate using np.datetime64
    time_coords = [np.datetime64(f'{int(buoy_year[i]):04d}-{int(buoy_month[i]):02d}-{int(buoy_day[i]):02d}T{int(buoy_hour[i]):02d}:{int(buoy_minute[i]):02d}')
                   for i in range(len(pId))]

    ds_buoy = xr.Dataset(
        {
            "sst": (("time"), buoy_sst),
            "lat": (("time"), buoy_lat),
            "lon": (("time"), buoy_lon),
            "year": (("time"), buoy_year),
            "month": (("time"), buoy_month),
            "day": (("time"), buoy_day),
            "hour": (("time"), buoy_hour),
            "minute": (("time"), buoy_minute),
            "pId": (("time"), pId),
        },
        coords={"time": time_coords},
    )

    fName_save = save_data_path + f'buoy_{buoy_ID}.nc'
    ds_buoy.to_netcdf(fName_save)
    print(f'Saved to: {fName_save}')

    return ds_buoy

# Inputs

In [ ]:
# Surface observations from iQuam
iquam_file = sorted(glob.glob("*.nc"))

obsType = 'buoy' # 'buoy' 'cMoor'
use_best_quality = True # Use certain quality of obs?
best_quality_value = 5 # Threshold for quality flag

target_year, target_month = [2025, 10]

lon_roi = [-90, -60.0]
lat_roi = [10, 25]
day_start, day_end = [21, 27]

# Zoom-region
lon_zoom_in = [-75, -67.5]
lat_zoom_in = [16, 20]

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/'

In [ ]:
# RTOFS from S3 bucket

BUCKET_NAME = 'noaa-nws-rtofs-pds'
KEY = 'rtofs.20251026/'
fname = 'rtofs_glo_3dz_n024_6hrly_hvr_US_east.nc'
rtofs_dStr = '2025-10-26T00'

download_s3_file(BUCKET_NAME, KEY, fname)
ds = xr.open_dataset(fname)

In [ ]:
# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im1=ds.temperature.isel(Depth=0).plot(x="Longitude", y="Latitude", transform=ccrs.PlateCarree(), vmin=28, vmax=34., cmap="bone_r",
cbar_kwargs={'shrink': 0.75})

for target_day in range(day_start, day_end):
  # observations of selected type and selected quality
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)

  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, target_year, target_month, target_day)

  im2=ax.scatter(lon, lat, transform=ccrs.PlateCarree(), s=16,\
                label=f'{target_year}/{target_month}/{target_day}')

ax.legend(loc=2, ncol=2)

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

ax.set_extent([lon_roi[0], lon_roi[1], lat_roi[0], lat_roi[1]])

ax.set_title(f'RTOFS SST valid for: {rtofs_dStr}')

# Same as above but with Salinity

In [ ]:
# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im1=ds.salinity.isel(Depth=0).plot(x="Longitude", y="Latitude", transform=ccrs.PlateCarree(), vmin=34, vmax=38., cmap="bone_r",
cbar_kwargs={'shrink': 0.75})

for target_day in range(day_start, day_end):
  # observations of selected type and selected quality
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)

  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, target_year, target_month, target_day)

  im2=ax.scatter(lon, lat, transform=ccrs.PlateCarree(), s=16,\
                label=f'{target_year}/{target_month}/{target_day}')

ax.legend(loc=2, ncol=2)

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

ax.set_extent([lon_roi[0], lon_roi[1], lat_roi[0], lat_roi[1]])

ax.set_title(f'RTOFS SSS valid for: {rtofs_dStr}')

## Zoom-in to see how close is RTOFS analysis SST to that from observations- a particular location/buoy

In [ ]:
cMin, cMax, cMap = [27, 31, "gist_ncar"]

# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im1=ds.temperature.isel(Depth=0).plot(x="Longitude", y="Latitude", transform=ccrs.PlateCarree(), vmin=cMin, vmax=cMax, cmap=cMap,
cbar_kwargs={'shrink': 0.75})

for target_day in range(day_start, day_end):
  # observations of selected type and selected quality
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)

  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, target_year, target_month, target_day)

  # target region
  #oType, year, month, day,hour, minute, pId, lat, lon, sst, qcFlag = filter_by_extent(lon_zoom_in[0], lon_zoom_in[1], lat_zoom_in[0], lat_zoom_in[1], oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag)

  # target region - NW of Jamaica
  oType, year, month, day,hour, minute, pId, lat, lon, sst, qcFlag = filter_by_extent(-70, -65, 20., 24., oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag)

  if target_day == day_start:
    found_buoy_ids = np.unique(pId)
    print(f'Found data from ID: {found_buoy_ids}')
    #buoy_ID = pId[0]

  im2=ax.scatter(lon, lat, transform=ccrs.PlateCarree(), c=sst-273.15, s=20,\
                 vmin=cMin, vmax=cMax, cmap=cMap,\
                 marker='s', edgecolor='none')

#ax.legend(loc=2, ncol=2)

# Add coastlines and land features
ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

ax.set_title(f'RTOFS SST valid for: {rtofs_dStr}')

#ax.set_extent([lon_zoom_in[0], lon_zoom_in[1], lat_zoom_in[0], lat_zoom_in[1]])
#ax.text(-69.5, 16.75,f"Drifting Buoy\nID: {buoy_ID}", transform=ccrs.PlateCarree(), fontsize=16)

ax.set_extent([-75, -65, 16., 24.])
ax.text(-69, 23,f"Drifting Buoy\nID:\n {found_buoy_ids}", transform=ccrs.PlateCarree(), fontsize=16)

In [ ]:
for target_day in range(day_start, day_end):
  # observations of selected type and selected quality
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)

  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag = filter_iquam_by_track_date(oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, target_year, target_month, target_day)

  # target region
  #oType, year, month, day,hour, minute, pId, lat, lon, sst, qcFlag = filter_by_extent(lon_zoom_in[0], lon_zoom_in[1], lat_zoom_in[0], lat_zoom_in[1], oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag)

  # target region - NW of Jamaica
  oType, year, month, day,hour, minute, pId, lat, lon, sst, qcFlag = filter_by_extent(-70, -65, 20., 24., oType, year, month, day, hour, minute, pId, lon, lat, sst, qcFlag)

  if target_day == day_start:
    found_buoy_ids = np.unique(pId)
    print(f'Found data from ID: {found_buoy_ids}')
    #buoy_ID = pId[0]

# SST time-series from a **specific** buoy(s) - that was found in the `zoom-in` region.

In [ ]:
for buoy_ID in found_buoy_ids:
  print(f'Processing data for buoy ID: {buoy_ID}')

  # Create a figure and axes with a PlateCarree projection
  oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
  load_iquam_data(iquam_file[0], obs_ids, obsType)

  if use_best_quality:
    oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag =\
    load_and_subset_iquam_data(obsType, oType, year, month, day, hour, minute, pId, lat, lon, sst, qcFlag, best_quality_value)

  mask = pId == buoy_ID

  buoy_year = year[mask]
  buoy_month = month[mask]
  buoy_day = day[mask]
  buoy_hour = hour[mask]
  buoy_minute = minute[mask]

  buoy_lat = lat[mask]
  buoy_lon = lon[mask]
  buoy_sst = sst[mask]

  create_and_save_buoy_dataset(buoy_year, buoy_month, buoy_day, buoy_hour, buoy_minute, buoy_lat, buoy_lon, buoy_sst, pId[mask], buoy_ID, save_data_path)

  #print(pId[mask])

In [ ]:
# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im = ax.scatter(buoy_lon, buoy_lat, transform=ccrs.PlateCarree(), c=buoy_sst-273.15, s=20, marker='s', edgecolor='none',
vmin=cMin, vmax=cMax, cmap=cMap)

ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

#ax.set_extent([lon_zoom_in[0], lon_zoom_in[1], lat_zoom_in[0], lat_zoom_in[1]])
#cbar_ax = fig.add_axes([0.4, 0.7, 0.25, 0.03])
ax.set_extent([-75, -60, 16., 24.])
cbar_ax = fig.add_axes([0.62, 0.75, 0.25, 0.03])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(f'ID: {buoy_ID} SST (°C)', fontsize=14)


# Time- series of the buoy SST.
buoy_dates = [f"{int(buoy_year[i]):04d}-{int(buoy_month[i]):02d}-{int(buoy_day[i]):02d}T{int(buoy_hour[i]):02d}:{int(buoy_minute[i]):02d}" for i in range(len(buoy_year))]

ax2 = fig.add_axes([0.18, 0.25, 0.28, 0.18])
ax2.plot(buoy_dates, buoy_sst - 273.15)
ax2.set_ylabel('SST (°C)')
ax2.set_xlabel(f'Day of {target_year}/{target_month}')

# Set x-axis ticks to show every 24 hours and format labels as day of the month
day_labels = [f"{int(day):02d}" for day in buoy_day[::24]]
ax2.set_xticks(buoy_dates[::24])
ax2.set_xticklabels(day_labels)
ax2.tick_params(axis='x', rotation=45)

In [ ]:
buoy_ID = 4101863
buoy = xr.open_dataset(f'/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/buoy_{buoy_ID}.nc')

buoy_lat = buoy.lat.values
buoy_lon = buoy.lon.values
buoy_sst = buoy.sst.values

buoy_year = buoy.year.values
buoy_month = buoy.month.values
buoy_day = buoy.day.values
buoy_hour = buoy.hour.values
buoy_minute = buoy.minute.values

In [ ]:
# Create a figure and axes with a PlateCarree projection
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im = ax.scatter(buoy_lon, buoy_lat, transform=ccrs.PlateCarree(), c=buoy_sst-273.15, s=20, marker='s', edgecolor='none',
vmin=cMin, vmax=cMax, cmap=cMap)

ax.coastlines(color='black', linewidth=1)
ax.add_feature(cfeature.LAND, edgecolor='black', alpha=0.75)

# Add gridlines
gl = ax.gridlines(draw_labels=True, zorder=0)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

#ax.set_extent([lon_zoom_in[0], lon_zoom_in[1], lat_zoom_in[0], lat_zoom_in[1]])
#cbar_ax = fig.add_axes([0.4, 0.7, 0.25, 0.03])
ax.set_extent([-75, -60, 16., 24.])
cbar_ax = fig.add_axes([0.62, 0.75, 0.25, 0.03])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(f'ID: {buoy_ID} SST (°C)', fontsize=14)


# Time- series of the buoy SST.
buoy_dates = [f"{int(buoy_year[i]):04d}-{int(buoy_month[i]):02d}-{int(buoy_day[i]):02d}T{int(buoy_hour[i]):02d}:{int(buoy_minute[i]):02d}" for i in range(len(buoy_year))]

ax2 = fig.add_axes([0.18, 0.25, 0.28, 0.18])
ax2.plot(buoy_dates, buoy_sst - 273.15)
ax2.set_ylabel('SST (°C)')
ax2.set_xlabel(f'Day of {target_year}/{target_month}')

# Set x-axis ticks to show every 24 hours and format labels as day of the month
day_labels = [f"{int(day):02d}" for day in buoy_day[::24]]
ax2.set_xticks(buoy_dates[::24])
ax2.set_xticklabels(day_labels)
ax2.tick_params(axis='x', rotation=45)